# Luke's invention Pt II: Find any route with unlimited stops

You are given a string like "UK:US:FedEx:4,UK:FR:Jet1:2,US:UK:RyanAir:8,CA:UK:CanadaAir:8" representing flights with from:to:airline:cost.

Given a source & destination, find any route to get from source to destination, with any number of stops. If there is no such route, return -1.

## Example 1:

**flights:** "UK:US:FedEx:4,UK:FR:Jet1:2,US:UK:RyanAir:8,CA:UK:CanadaAir:8"
**Input:** US, FR
**Output:** "US:UK:FR", 10

## Constraints:

- src != dst

In [15]:
from collections import defaultdict, deque
from functools import cache

# Bellman-Ford BFS approach
class Solution():

    def __init__(self, flights: str):
        self.flightDirectory = self.getFlightDirectory(flights)

    def getFlightDirectory(self, flights: str) -> dict[dict]:
        # "UK:US:FedEx:4,UK:FR:Jet1:2,US:UK:RyanAir:8,CA:UK:CanadaAir:8"
        flightDirectory = defaultdict(lambda: defaultdict(list))
        flightList = flights.split(',')
        for flight in flightList:
            src, dst, airline, cost = flight.split(':')
            flightDirectory[src][dst].append({
                'airline': airline,
                'cost': float(cost)
            })
        return flightDirectory
    
    def findCheapestFlight(self, src: str, dst: str) -> str:
        minCost = float('inf')
        minCostFlight = None
        for flight in self.flightDirectory[src][dst]:
            if flight['cost'] < minCost:
                # minCostFlight = f"{src}:{dst}:{flight['airline']}:{flight['cost']}"
                minCostFlight = flight
        return minCostFlight

    def findRoute(self, src: str, dst: str) -> str:
        # Use BFS to minimize the number of stops
        queue = deque([(src, 0)])
        visited = set([src])
        parent = {src: None}

        while queue:
            v, cost = queue.popleft()
            visited.add(v)

            if v == dst:
                # return "US:UK:FR", 10
                curr, path = dst, []
                while curr:
                    path.append(curr)
                    curr = parent[curr]
                path = path[::-1]
                return (':'.join(path), cost)
            
            for adj in self.flightDirectory[v]:
                flight = self.findCheapestFlight(v, adj)
                if adj not in visited:
                    queue.append((adj, cost + flight['cost']))
                    parent[adj] = v
        
        return None


In [16]:
# TEST CASES

sol1 = Solution("UK:US:FedEx:4,UK:FR:Jet1:2,US:UK:RyanAir:8,CA:UK:CanadaAir:8")

result = sol1.findRoute('US', 'FR')

assert(result == ('US:UK:FR', 10))
